## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [ ]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [ ]:
import numpy as np
import mip
m = mip.Model()

#SETS
T =   [i for i in range(8)]
T0 =  [i for i in range(-1, 8)]
F =   [i for i in range(5)]

#PARAMETERS
d = [ [   700,    1500,   700,    0,      0,    700,    1500,   0],
      [   1350,   0,      450,    0,      1350, 0,      450,    0],
      [   0,      1500,   1500,   0,      0,    1500,   1500,   0],
      [   820,    1560,   820,    0,      820,  1560,   820,    0],
      [   0,      680,    1080,   0,      0,    680,    1080,   0]]

c =   [   0.2,    0.18,   0.19,   0.21,   0.23  ]
s =   [   0.002,  0.012,  0.007,  0.011,  0.015 ]
dm =  [   15,     28,     35,     37,     42    ]

dmpd = 20
dmpu = 40
kWhpt = 421.6
ppkWh = 0.28
maxeprod = 1950000
maxsc = 500

#VARIABLES
x = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T}
r = {(i,t) : m.add_var(var_type=mip.CONTINUOUS) for i in F for t in T0}


#CONSTAINTS

#for each quarter t and for each farm i the corn quantity bought is less than or equal than the quantity offered by the farm
for i in F:
  for t in T:
    m.add_constr(x[i, t] <= d[i][t])

#for each quarter t and for each farm i the corn quantity stored is less than or equal than the quantity bought plus the quantity stored previouly
for i in F:
  for t in T:
    m.add_constr(r[i, t] <= x[i, t] + r[i, t-1])